In [2]:
import pandas as pd
import numpy as np
import requests

In [6]:
df = pd.read_csv('coordinates.csv')

In [7]:
df

,Name,Latitude,Longitude
0,WPI,42.274600,-71.806300
1,Umass Medical,42.277200,-71.761100
2,Worcester State,42.268000,-71.844100
3,Holy Cross,42.239200,-71.808000
4,Clark,42.252300,-71.824700
5,Assumption,42.291500,-71.829600
6,Quinsigamond,42.314600,-71.793700
7,Becker,42.267000,-71.811900
8,MCPHS,42.336800,-71.101200
9,Worcester Common,42.262600,-71.802300
